In [1]:
import pandas as pd
import numpy as np
import re
import spacy
from franc_lib import lexical


normalizer = lexical.Preprocessing()

In [2]:
dataframe = pd.read_csv('dados.csv')

In [3]:
text = dataframe[dataframe['Unnamed: 0'] == 2]['text']

In [4]:
# ([^rs])  - Qualquer letra que não r ou s
# (?=\1+)  - Que se repita uma vez ou mais
# |(rr)    - Ou dois r's
# (?=r+)   - Que tenham mais r's à frente
# |(ss)    - Ou dois s's
# (?=s+)   - Que tenham mais s's à frente
regex = r"([^rs])(?=\1+)|(rr)(?=r+)|(ss)(?=s+)"

def remove_username(tweet):
    return re.sub('@[^\s]+','',tweet)

def remove_end_of_line(tweet):
    return tweet.replace('\n', '').replace('RT', '')

def remove_duplicate_letters(tweet):
    tweet = re.sub(regex, '', tweet, 0)
    return tweet

In [5]:
dataframe.head()

,Unnamed: 0,id,text
0,0,1142208313532342274,Nossa putaria é quase um poema
1,1,1142208269513089024,ManooooKKKKKKKKK apareceu aqueles vídeos de li...
2,2,1142208264861626370,@felipesimoncini É putaria né ? Aqui no dique ...
3,3,1142208231558864896,acabei de fazer um Tinder pra minha tia e fale...
4,4,1142208138520776704,RT @MiiEDB: Queria viver um romance porém esta...


In [6]:
def normalize(tweet):
    tweet = remove_username(tweet)
    tweet = remove_end_of_line(tweet)
    tweet = remove_duplicate_letters(tweet)
    tweet = normalizer.lowercase(tweet)
    tweet = normalizer.remove_punctuation(tweet)
    tokens = normalizer.tokenize_words(tweet)
#     tokens = [token for token in tokens if token not in english_stopwords]
    return ' '.join(tokens)

In [7]:
dataframe['normalize'] = dataframe['text'].apply(normalize)

In [8]:
dataframe.head()

,Unnamed: 0,id,text,normalize
0,0,1142208313532342274,Nossa putaria é quase um poema,nossa putaria é quase um poema
1,1,1142208269513089024,ManooooKKKKKKKKK apareceu aqueles vídeos de li...,manok apareceu aqueles vídeos de ligações do b...
2,2,1142208264861626370,@felipesimoncini É putaria né ? Aqui no dique ...,é putaria né aqui no dique é frenquentar nos a...
3,3,1142208231558864896,acabei de fazer um Tinder pra minha tia e fale...,acabei de fazer um tinder pra minha tia e fale...
4,4,1142208138520776704,RT @MiiEDB: Queria viver um romance porém esta...,queria viver um romance porém estamos no sécul...


In [9]:
hate_certeiras = [
    'feminazi','gayzista', 'viadismo', 'homossexualismo', 'viadagem', 'gayzismo', 'favelado',
    'gayzinho', 'sapatona', 'sapatão', 'bixa', 'verme', 'negrice', 'negrisse', 'negraiada','abortistas',
    'rameira', 'crioula', 'crioulas', 'crioulos', 'crioulo','tições', 'sangues ruins', 'Bambis', 'boiola',
    'bichona', 'biba', 'traveco', 'travesti'
]

hate_verbo = [
    'matar', 'exterminar', 'odiar', 'queimar', 'atirar', 'esfaquear', 'bater', 'apanhar', 'roubar',
    'estuprar', 'ser', 'comer',
]

hate_adj = [
    'safado','imbecil', 'imbecis', 'cachaceiro', 'babaca', 'idiota', 'puta', 'puto', 'arrombado','arrombada',
    'canalha', 'vagabundo', 'vagabunda','burra', 'burro', 'maldito', 'maldita', 'neguim', 'neguinho', 
    'favelados', 'faveladas', 'favelada', 'maconheiro', 'maconhero', 'viado'
]

hate_words = [
    'putaria','bosta','cuzao', 'fdp', 'filho da puta',
    'merda', 'escroto', 'transfobia', 'gordo', 'canalha',
    'ditadura','gorda'
]

In [10]:
# Modelo ja baixado
# import subprocess
# command = "python3 -m spacy download pt_core_news_sm".split()
# subprocess.call(command)

nlp = spacy.load('pt_core_news_sm')

def tag(tweet):
    for hate in hate_certeiras:
        if hate in tweet:
            return 1
    
    doc = nlp(tweet)
    
    for token in doc:
        if token.pos_ in ['VERB', 'AUX', 'DET']:
            if token.lemma_ in hate_verbo:
                for hate in hate_adj:
                    if hate in tweet:
                        return 1
                for hate in hate_words:
                    if hate in tweet:
                        return 1
            elif token.lemma_ in hate_adj:
                return 1
#         elif token.text == 'não' or token.dep_ == 'advmod':
#                 for hate in ['gostar', 'ser', 'amar']:
#                     for token2 in doc:
#                         if token2.lemma_ == hate:
#                             return 1
#                 for hate in hate_adj:
#                     if hate in tweet:
#                         return 0
#                 for hate in hate_words:
#                     if hate in tweet:
#                         return 0
        elif token.lemma_ in hate_adj:
            return 1
    return 0

def split(probability=0.8):
    if probability < np.random.rand():
        return 'train'
    return 'test'
    

In [11]:
my_set = []
i = 0
for i in range(0, len(dataframe['text'])):
    my_set.append(split())
dataframe['set'] = my_set

dataframe['tag'] = dataframe['normalize'].apply(tag)

In [12]:
dataframe.head()

,Unnamed: 0,id,text,normalize,set,tag
0,0,1142208313532342274,Nossa putaria é quase um poema,nossa putaria é quase um poema,train,1
1,1,1142208269513089024,ManooooKKKKKKKKK apareceu aqueles vídeos de li...,manok apareceu aqueles vídeos de ligações do b...,test,1
2,2,1142208264861626370,@felipesimoncini É putaria né ? Aqui no dique ...,é putaria né aqui no dique é frenquentar nos a...,test,1
3,3,1142208231558864896,acabei de fazer um Tinder pra minha tia e fale...,acabei de fazer um tinder pra minha tia e fale...,test,0
4,4,1142208138520776704,RT @MiiEDB: Queria viver um romance porém esta...,queria viver um romance porém estamos no sécul...,test,0


In [45]:
dataframe.groupby('tag').count()

,Unnamed: 0,id,text,normalize,set
tag,,,,,
0,48188,48188,48188,48188,48188
1,36993,36993,36993,36993,36993


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
train_reviews = dataframe[dataframe['set'] == 'train']['normalize'].values.tolist()
train_classes = dataframe[dataframe['set'] == 'train']['tag'].values.tolist()

test_reviews = dataframe[dataframe['set'] == 'test']['normalize'].values.tolist()
test_classes = dataframe[dataframe['set'] == 'test']['tag'].values.tolist()

transformer = TfidfVectorizer()
transformer.fit(train_reviews)
X = transformer.transform(train_reviews)
X_test = transformer.transform(test_reviews)

In [25]:
from sklearn.linear_model import LogisticRegression

In [26]:
classifier_lr = LogisticRegression(n_jobs=3)
classifier_lr.fit(X, train_classes)

/home/franciscone/.envs/tp-nlp/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/franciscone/.envs/tp-nlp/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=3, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [55]:
sentence = "morra travesti"
instance = transformer.transform([normalize(sentence)])
classifier_lr.predict(instance)

array([0])

In [28]:
from sklearn.metrics import accuracy_score

In [32]:
accuracy_score(test_classes, classifier_lr.predict(X_test))

0.9503774194495844

In [20]:
for token in doc:
    print(token.text, token.lemma_)

Sou Sou
feliz feliz
gay gay
